# Document a California Housing Price Prediction regression model

Use the [California Housing Price Prediction](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) sample dataset from Sklearn to train a simple regression model and document that model with the ValidMind Library.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Library
- Loading a sample dataset provided by the library to train a simple regression model
- Running a ValidMind test suite to quickly generate documention about the data and model

::: {.content-hidden when-format="html"}
## Contents    
- [About ValidMind](#toc1__)    
- [Before you begin](#toc2__)    
- [Setting up](#toc3__)    
  - [Install the ValidMind Library](#toc3_1__)    
  - [Initialize the ValidMind Library](#toc3_2__)    
    - [Register sample model](#toc3_2_1__)    
    - [Apply documentation template](#toc3_2_2__)    
    - [Get your code snippet](#toc3_2_3__)    
  - [Initialize the Python environment](#toc3_3__)    
  - [Preview the documentation template](#toc3_4__)    
- [Load the sample dataset](#toc4__)    
- [Document the model](#toc5__)    
  - [Prepocess the raw dataset](#toc5_1__)    
  - [Initialize the ValidMind datasets](#toc5_2__)    
  - [Initialize a model object](#toc5_3__)    
  - [Assign predictions to the datasets](#toc5_4__)    
  - [Run the full suite of tests](#toc5_5__)    
- [Next steps](#toc6__)    
- [Upgrade ValidMind](#toc7__)    

:::
<!-- jn-toc-notebook-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/jn-toc-notebook-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1__'></a>

## About ValidMind

ValidMind's suite of tools enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/get-started/get-started.html) — The basics, including key concepts, and how our products work
- [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html) — The path for developers, more code samples, and our developer reference

<a id='toc2__'></a>

## Before you begin

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc3__'></a>

## Setting up

<a id='toc3_1__'></a>

### Install the ValidMind Library

To install the library:

In [ ]:
%pip install -q validmind

<a id='toc3_2__'></a>

### Initialize the ValidMind Library

<a id='toc3_2_1__'></a>

#### Register sample model

Let's first register a sample model for use with this notebook:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and click **+ Register Model**.

3. Enter the model details and click **Next >** to continue to assignment of model stakeholders. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

   For example, to register a model for use with this notebook, select the following use case: `Marketing/Sales - Attrition/Churn Management`

4. Select your own name under the **MODEL OWNER** drop-down.

5. Click **Register Model** to add the model to your inventory.

<a id='toc3_2_2__'></a>

#### Apply documentation template

Once you've registered your model, let's select a documentation template. A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

1. In the left sidebar that appears for your model, click **Documents** and select **Documentation**.

2. Under **TEMPLATE**, select `Binary classification`.

3. Click **Use Template** to apply the template.

<a id='toc3_2_3__'></a>

#### Get your code snippet

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

1. On the left sidebar that appears for your model, select **Getting Started** and click **Copy snippet to clipboard**.
2. Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

<a id='toc3_3__'></a>

### Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

%matplotlib inline

<a id='toc3_4__'></a>

### Preview the documentation template

Let's verify that you have connected the ValidMind Library to the ValidMind Platform and that the appropriate *template* is selected for your model.

You will upload documentation and test results unique to your model based on this template later on. For now, **take a look at the default structure that the template provides with [the `vm.preview_template()` function](https://docs.validmind.ai/validmind/validmind.html#preview_template)** from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

<a id='toc4__'></a>

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
# Import the sample dataset from the library

from validmind.datasets.regression import california_housing as demo_dataset

print(f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}")

raw_df = demo_dataset.load_data()
raw_df.head()

<a id='toc5__'></a>

## Document the model

As part of documenting the model with the ValidMind Library, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.

<a id='toc5_1__'></a>

### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize RandomForestRegressor regressor: Creates an `RandomForestRegressor` object with random state set to 0.
- Set evaluation metrics: Specifies metrics for model evaluation as "errors" and "r2".
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

Here we create two regression models so that the performance of the model can be compared through ValidMind test suite.

In [ ]:
scale = False
if scale:
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.fit_transform(x_val)

model = RandomForestRegressor(random_state=0)
model.fit(x_train, y_train)
s1 = model.score(x_train, y_train)
s2 = model.score(x_val, y_val)
print("R² of Support Vector Regressor on training set: {:.3f}".format(s1))
print("R² of Support Vector Regressor on test set: {:.3f}".format(s2))

model_1 = GradientBoostingRegressor(random_state=0, max_depth=4)
model_1.fit(x_train, y_train)
model1_s1 = model_1.score(x_train, y_train)
model1_s2 = model_1.score(x_val, y_val)
print(
    "R² of Support Gradient Boosting Regressor on training set: {:.3f}".format(
        model1_s1
    )
)
print("R² of Support Gradient Boosting Regressor on test set: {:.3f}".format(model1_s2))

<a id='toc5_2__'></a>

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=demo_dataset.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df, input_id="train_dataset", target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, input_id="test_dataset", target_column=demo_dataset.target_column
)

<a id='toc5_3__'></a>

### Initialize a model object

Additionally, you need to initialize a ValidMind model objects (`vm_model` and `vm_model_1`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="random_forest_regressor",
)
vm_model_1 = vm.init_model(
    model_1,
    input_id="gradient_boosting_regressor",
)

<a id='toc5_4__'></a>

### Assign predictions to the datasets

We can now use the assign_predictions() method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)
vm_train_ds.assign_predictions(
    model=vm_model_1,
)
vm_test_ds.assign_predictions(
    model=vm_model,
)
vm_test_ds.assign_predictions(
    model=vm_model_1,
)

<a id='toc5_5__'></a>

### Run the full suite of tests

This is where it all comes together: you are now ready to run the documentation tests for the model as defined by the documentation template you looked at earlier.

The [`vm.run_documentation_tests`](https://docs.validmind.ai/validmind/validmind.html#run_test_suite) function finds and runs every tests specified in the test suites and then uploads all the documentation and test artifacts that get generated to the ValidMind Platform.

The function takes two arguments:

- `dataset`: The data to be tested, specified as `vm_dataset`.
- `model`: The candidate model to be used for testing, specified as `vm_model`. -`models`: The list of models that can be compare with candidate model.

The variable `full_suite` then holds the result of these tests.

In [ ]:
full_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_train_ds,
        "datasets": (vm_train_ds, vm_test_ds),
        "model": vm_model,
        "models":[vm_model_1]
    }
)

<a id='toc6__'></a>

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform:

1. In the ValidMind Platform, click **Documentation** under Documents for the model you registered earlier. ([Need more help?](https://docs.validmind.ai/guide/model-documentation/working-with-model-documentation.html)

2. Expand the following sections and take a look around:

   - **2. Data Preparation**
   - **3. Model Development**

What you can see now is a much more easily consumable version of the documentation, including the results of the tests you just performed, along with other parts of your model documentation that still need to be completed. There is a wealth of information that gets uploaded when you run the full test suite, so take a closer look around, especially at test results that might need attention (hint: some of the tests in **2.1 Data description** look like they need some attention).

If you want to learn more about where you are in the model documentation process, take a look our documentation on the [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html).

<a id='toc7__'></a>

## Upgrade ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;">After installing ValidMind, you’ll want to periodically make sure you are on the latest version to access any new features and other enhancements.</div>

Retrieve the information for the currently installed version of ValidMind:

In [ ]:
%pip show validmind

If the version returned is lower than the version indicated in our [production open-source code](https://github.com/validmind/validmind-library/blob/prod/validmind/__version__.py), restart your notebook and run:

```bash
%pip install --upgrade validmind
```

You may need to restart your kernel after running the upgrade package for changes to be applied.

<!-- VALIDMIND COPYRIGHT -->

<small>

***

Copyright © 2023-2026 ValidMind Inc. All rights reserved.<br>
Refer to [LICENSE](https://github.com/validmind/validmind-library/blob/main/LICENSE) for details.<br>
SPDX-License-Identifier: AGPL-3.0 AND ValidMind Commercial</small>